In [1]:
###Analyze LTS for edges
#The process is divided in different steps:
# 1) biking allowed, not allowed
# 2) check if there are separated paths and assign lowest lst value
# 3) check for separated paths and unseparated
# 4) check the presence of bike lanes in the unseparated scenario
# 5) check presence or not of parking in bike lanes
# 6) mixed traffic analyisis
from ltsfunctions import BikePathAnalysis
import json
import pandas as pd
import pickle
import osmnx as ox
import os

# Load the GeoDataFrames from the specified path
pickle_path = "/Users/leonardo/Desktop/Tesi/LTSBikePlan/data/gdf_data.pkl"
with open(pickle_path, 'rb') as f:
    gdf_nodes, gdf_edges, city = pickle.load(f)

# Start by finding where biking is allowed and get edges where biking is not *not* allowed.
print("The shape of gdf edges:", gdf_edges.shape)
print(gdf_edges.columns)
gdf_allowed, gdf_not_allowed = BikePathAnalysis.biking_permitted(gdf_edges)

print("The shape of gdf edges allowed:",gdf_allowed.shape)
print("The shape of gdf edges not allowed:",gdf_not_allowed.shape)

# check for separated path
separated_edges, unseparated_edges = BikePathAnalysis.is_separated_path(gdf_allowed)
# assign separated ways lts = 1
separated_edges = separated_edges.copy()
separated_edges.loc[:, 'lts'] = 1
print("The shape of separated edges:", separated_edges.shape)
print("The shape of unseparated edges:", unseparated_edges.shape)

#check the presence of bike lane in unseparated paths
to_analyze, no_lane = BikePathAnalysis.is_bike_lane(unseparated_edges)
print(to_analyze)
print("The shape of analyze lanes:",to_analyze.shape)
print("The shape of no lane:",no_lane.shape)

#check presence of the parking or not
parking_detected, parking_not_detected = BikePathAnalysis.parking_present(to_analyze)
print("The shape of parking_detected:",parking_detected.shape)
print("The shape of parking no-detected:",parking_not_detected.shape)
parking_lts = BikePathAnalysis.bike_lane_analysis_with_parking(parking_detected)
no_parking_lts = BikePathAnalysis.bike_lane_analysis_without_parking(parking_not_detected)

# Next - mixed traffic analysis
lts_no_lane = BikePathAnalysis.mixed_traffic(no_lane)
print("The shape of lts_no_lane:",lts_no_lane.shape)

# final components: lts_no_lane, parking_lts, no_parking_lts, separated_edges should all add up
# these should all add up to gdf_allowed
# print(gdf_allowed.shape)
lts_no_lane.shape[0] + parking_lts.shape[0] + no_parking_lts.shape[0] + separated_edges.shape[0]
gdf_not_allowed['lts'] = 0
all_lts = pd.concat([separated_edges, parking_lts, no_parking_lts, lts_no_lane, gdf_not_allowed])
print("The shape of all_lts:", all_lts.shape)

# Apply the slope_penalty function to all_lts
all_lts = BikePathAnalysis.slope_penalty(all_lts)

# decision rule glossary (taken from Bike Ottawa)
# Load the dictionaries from the JSON file
with open('LTS_decisionrule_dict.json', 'r') as file:
    data = json.load(file)
    rule_message_dict = data['rule_message_dict']
    simplified_message_dict = data['simplified_message_dict']

# Use the dictionaries in your code
all_lts['message'] = all_lts['rule'].map(rule_message_dict)
all_lts['short_message'] = all_lts['rule'].map(simplified_message_dict)

print(all_lts.columns)


The shape of gdf edges: (7736, 22)
Index(['osmid', 'reversed', 'name', 'oneway', 'length', 'bridge', 'maxspeed',
       'tunnel', 'highway', 'ref', 'lanes', 'junction', 'access', 'service',
       'width', 'landuse', 'est_width', 'context', 'group', 'slope',
       'slope_class', 'geometry'],
      dtype='object')
The shape of gdf edges allowed: (7720, 23)
The shape of gdf edges not allowed: (16, 23)
The shape of separated edges: (1638, 24)
The shape of unseparated edges: (6082, 22)
Empty GeoDataFrame
Columns: [osmid, reversed, name, oneway, length, bridge, maxspeed, tunnel, highway, ref, lanes, junction, access, service, width, landuse, est_width, context, group, slope, slope_class, geometry]
Index: []

[0 rows x 22 columns]
The shape of analyze lanes: (0, 22)
The shape of no lane: (6082, 22)
The shape of parking_detected: (0, 22)
The shape of parking no-detected: (0, 22)
The shape of lts_no_lane: (6082, 26)
The shape of all_lts: (7736, 26)
Index(['osmid', 'reversed', 'name', 'oneway'

In [2]:
# Calculate node LTS. 
# - An intersection without either was assigned the highest LTS of its intersecting roads. 
# - Stop signs reduced an otherwise LTS2 intersection to LTS1. 
# - A signalized intersection of two lowstress links was assigned LTS1. 
# - Assigned LTS2 to signalized intersections where a low-stress (LTS1/ 2) link crosses a high-stress (LTS3/4) link. 
# Apply the function to the dataframe
gdf_nodes['lts'], gdf_nodes['message'] = zip(*gdf_nodes.apply(BikePathAnalysis.calculate_lts_nodes, args=(all_lts,), axis=1))

In [3]:
import os
import osmnx as ox
import pandas as pd
import geopandas as gpd
import xml.etree.ElementTree as ET
import uuid

def save_and_correct_graphml(G, filepath):
    # Temporarily save the graph to a .graphml file
    temp_path = filepath + "_temp.graphml"
    ox.save_graphml(G, temp_path)

    # Load the temporary graphml as an xml tree
    tree = ET.parse(temp_path)
    root = tree.getroot()
    ns = {'graphml': 'http://graphml.graphdrawing.org/xmlns'}
    
    # Iterate through the data elements and handle floats that should be integers
    for data in root.findall(".//graphml:data", ns):
        if data.text and data.text.endswith(".0"):  # This will match strings like "258916000.0"
            try:
                integer_value = int(float(data.text))
                data.text = str(integer_value)
            except:
                pass

        # If osmid is NaN or missing, generate a unique positive value
        if data.text == "nan" or data.text is None:
            data.text = str(abs(uuid.uuid4().int))

    # Remove the namespace prefix
    for elem in root.iter():
        elem.tag = elem.tag.split('}')[-1]

    # Set the correct xmlns attribute for the root element
    root.attrib["xmlns"] = "http://graphml.graphdrawing.org/xmlns"
    if "xmlns:ns0" in root.attrib:
        del root.attrib["xmlns:ns0"]

    # Save the corrected graphml and remove the temporary one
    tree.write(filepath, xml_declaration=True, encoding='utf-8', method="xml")
    os.remove(temp_path)


base_path = "/Users/leonardo/Desktop/Tesi/LTSBikePlan/data/"
def extract_city(citta):
    # Splitting by comma and taking the first part
    first_part = citta.split(",")[0]

    # Handling cases where the city name might have a hyphen (like 'Bolzano - Bozen')
    city_name = first_part.split("-")[0].strip()

    # Replacing spaces with underscores after handling the hyphen
    city_sanitized = city_name.replace(" ", "_")

    return city_sanitized

# Sanitize the city name
city_sanitized = extract_city(city)

# Save nodes data
gdf_nodes.to_csv(f"{base_path}{city_sanitized}_gdf_nodes.csv")

# Save a subset of all_lts directly
all_lts[['osmid', 'lanes', 'name', 'highway', 'maxspeed', 'geometry', 'length', 'rule', 'lts', 'group', 'slope', 'slope_class',
         'lanes_assumed', 'maxspeed_assumed', 'message', 'short_message']].to_csv(f"{base_path}{city_sanitized}_all_lts.csv")

# Ensure that osmid is numeric
all_lts['osmid'] = all_lts['osmid'].apply(pd.to_numeric, errors='coerce')

# Convert osmid values to avoid scientific notation
all_lts['osmid'] = all_lts['osmid'].apply(lambda x: "{:.0f}".format(x) if isinstance(x, (int, float)) else x)

# # Check and notify if conversion was successful
if all_lts['osmid'].astype(str).str.contains('e\+', regex=True).any():
    print(f"OSMID column still contains values in scientific notation format.")
else:
    print("OSMID column successfully handled!")

# Convert back to GeoDataFrame and set CRS
all_lts = gpd.GeoDataFrame(all_lts, geometry='geometry')
all_lts.crs = "EPSG:4326"

# Create the graph
G_lts = ox.graph_from_gdfs(gdf_nodes, all_lts[['osmid', 'lanes', 'name', 'highway',  'maxspeed', 'geometry', 'length', 'rule', 'lts', 
                                               'slope', 'slope_class', 'lanes_assumed', 'maxspeed_assumed', 'message', 'short_message']])

# Save the graph using the new function
# ox.save_graphml(G_lts, f"{base_path}{city_sanitized}_lts.graphml")
# Save the graph using the new function
save_and_correct_graphml(G_lts, f"{base_path}{city_sanitized}_lts.graphml")

# Remove the pickle file after processing
os.remove(pickle_path)

OSMID column successfully handled!
